In [2]:
import pandas as pd
import numpy as np
import re
import string
from konlpy.tag import Komoran
import torch
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\train.csv"
train = pd.read_csv(file_path)

file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\test.csv"
test = pd.read_csv(file_path)

file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\sample_submission.csv"
sample = pd.read_csv(file_path)

In [ ]:
from konlpy.tag import Komoran

komoran = Komoran()

with open('C:/Users/82106/Desktop/데이콘 한솔데크/불용어.txt', 'r', encoding='cp949') as f:
  list_file = f.readlines() 
stopwords = list_file[0].split(",")

# 정규화
def preprocess(text):
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', ' ', str(text).strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text


# 명사/영단어 추출, 한글자 제외, 불용어 제거
def remove_stopwords(text):
    n = []
    word = komoran.nouns(text)
    p = komoran.pos(text)
    for pos in p:
      if pos[1] in ['SL']:
        word.append(pos[0])
    for w in word:
      if len(w)>1 and w not in stopwords:
        n.append(w)
    return " ".join(n)

# 최종
def finalpreprocess(text):
  return remove_stopwords(preprocess(text))

# 전처리 및 불용어 제거 적용 (두 단계)
train["재발방지대책 및 향후조치계획_preprocessed"] = train["재발방지대책 및 향후조치계획"].apply(finalpreprocess)

# 결과 확인
print(train["재발방지대책 및 향후조치계획_preprocessed"].head())

In [10]:
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 로드
model = SentenceTransformer("jhgan/ko-sbert-sts")
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
llm_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [ ]:
from sklearn.cluster import KMeans

# 인적사고 그룹화
grouped = train.groupby("인적사고")

res = {}

for name, group in tqdm(grouped):
    plans = group["재발방지대책 및 향후조치계획"].dropna().tolist()

    if len(plans) == 0:
        continue  # 대책 텍스트가 없는 경우 건너뛰기

    # 문장 임베딩 생성
    vectors = model.encode(plans, batch_size=32, show_progress_bar=True)

    # 최적 클러스터 개수 c설정 (문장 수에 따라 다르게 적용)
    num_clusters = min(len(plans), 3)  # 최소 3개의 클러스터 (문장 수보다 많을 수 없음)

    if num_clusters > 1:
        # KMeans 클러스터링 수행
        kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
        kmeans.fit(vectors)
        cluster_centers = kmeans.luster_centers_

        # 각 클러스터별 대표 문장 선정 (중심과 가장 가까운 문장)
        representative_indices = []
        for center in cluster_centers:
            similarities = cosine_similarity([center], vectors)[0]
            best_idx = similarities.argmax()
            representative_indices.append(best_idx)

        representative_plans = [plans[i] for i in representative_indices]
    else:
        # 문장이 하나밖에 없는 경우 그대로 사용
        representative_plans = plans

    # 최종 대표 문장 선정 (유사도 기준 최선의 하나만 선택)
    final_vectors = model.encode(representative_plans)
    final_similarities = cosine_similarity(final_vectors, final_vectors)
    best_idx = final_similarities.mean(axis=1).argmax()
    representative_plan = representative_plans[best_idx]

    res[name] = representative_plan

# 결과 확인
print(res)


  0%|          | 0/23 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

{'감전': '현장대리인 및 공사감리자 안전교육 및 후속점검 예정.', '교통사고': '작업용 차량의 사전작업계획서 제출, 운전원 안전교육 실시, 전담신호수 배치 및 근로자 접근통제 조치 준수, 정기적인 재발 방지 이행 여부 확인과 현장 안전 점검 실시.', '기타': '작업전 안전교육 및 특별안전교육 실시와 안전관리 및 안전교육 철저를 통한 재발 방지 대책.', '깔림': '작업전 안전교육 및 안전점검 철저를 통한 사고 재발 방지 대책.', '끼임': '안전작업계획 및 작업내용 주의와 교육 실시와 작업 전 안전점검 실시를 통한 재발 방지 대책.', '넘어짐(기타)': '작업자 교육 실시와 현장 안전관리 철저를 통한 안전사고 예방.', '넘어짐(물체에 걸림)': '작업 이동 동선 확보, 돌출 자재 보호 조치 실시 및 표식 설치, 현장 전 근로자 특별안전교육 실시와 안전수칙 이행 여부 관리감독 철저.', '넘어짐(미끄러짐)': '작업 시작 전 안전교육 실시 및 법정 안전교육 강화, 현장 위험 요인 수시 점검, 위험 부분 정리정돈 철저, 사전 안내 시설 설치를 통한 작업자의 안전의식 고취.', '떨어짐(10미터 이상)': '안전 교육 실시와 작업자 특별안전교육 및 현장안전관리 철저를 통한 사고 재발 방지 대책.', '떨어짐(2미터 미만)': '작업방법 및 순서에 대한 근로자 안전교육 실시와 T.B.M 시 안전교육 강화, 관리감독자의 수시 점검, 현장 내 작업 전후 안전교육 철저 진행, 근로자 작업구간 수시 순찰 실시를 통한 재해 발생 방지 관리감독 강화.', '떨어짐(2미터 이상 ~ 3미터 미만)': '안전수칙 철저 및 작업자 안전교육 실시를 통한 재발 방지 대책과 향후 조치 계획.', '떨어짐(3미터 이상 ~ 5미터 미만)': '사고 예방을 위한 안전조치 및 작업자 안전교육 실시와 작업현장 관리 철저.', '떨어짐(5미터 이상 ~ 10미터 미만)': '낙하물 방지 안전망 설치와 작업자 안전장치 사용 철저, 공사중지 후 현장 안전시설 추가 보완 및 작업자 안전교육

In [13]:
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

In [14]:
for i in range(len(test)):
    accident = test.loc[i, "인적사고"]
    sample.loc[i, "재발방지대책 및 향후조치계획"] = res[accident]
    sample.iloc[i, 2:] = res_v[accident]

In [15]:
sample.to_csv("C:/Users/82106/Desktop/데이콘 한솔데크/data/result/baseline+RAG+Clustering.csv", index=False, encoding='utf-8-sig')